# All decoders (except KF, NB, and ensemble) run with variable bin sizes

## User Options

Define what folder you're saving to

In [1]:
# save_folder=''
save_folder='/home/jglaser/Files/Neural_Decoding/Results/'

Define what folder you're loading the files from

In [2]:
# load_folder=''
load_folder='/home/jglaser/Data/DecData/'

Define what dataset you are using

In [3]:
# dataset='s1'
# dataset='m1'
dataset='hc'

Define which decoder to run

In [4]:
run_wf=1 #Wiener Filter
run_wc=0 #Wiener Cascade
run_svr=0 #Support vector regression
run_xgb=0 #XGBoost
run_dnn=0 #Feedforward (dense) neural network
run_rnn=0 #Recurrent neural network
run_gru=0 #Gated recurrent units
run_lstm=0 #Long short term memory network

## 1. Import Packages

We import both standard packages, and functions from the accompanying .py files

In [5]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle
import time
import sys

#Add the main folder to the path, so we have access to the files there.
#Note that if your working directory is not the Paper_code folder, you may need to manually specify the path to the main folder. For example: sys.path.append('/home/jglaser/GitProj/Neural_Decoding')
sys.path.append('..') 

#Import function to get the covariate matrix that includes spike history from previous bins
from preprocessing_funcs import get_spikes_with_history

###Import functions for binning data for preprocessing###
from preprocessing_funcs import bin_spikes
from preprocessing_funcs import bin_output

#Import metrics
from metrics import get_R2
from metrics import get_rho

#Import decoder functions
from decoders import WienerCascadeDecoder
from decoders import WienerFilterDecoder
from decoders import DenseNNDecoder
from decoders import SimpleRNNDecoder
from decoders import GRUDecoder
from decoders import LSTMDecoder
from decoders import XGBoostDecoder
from decoders import SVRDecoder

#Import Bayesian Optimization package
from bayes_opt import BayesianOptimization

/opt/anaconda/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 45.0% of memory, cuDNN Mixed dnn version. The header is from one version, but we link with a different version (5103, 5110))
/opt/anaconda/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be t

In [6]:
#Turn off deprecation warnings

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## 2. Load Data

Here, we load data in the more raw format, since the preprocessing depends on the bin size (which we will vary later on)

In [7]:
if dataset=='s1':    
    data=io.loadmat(load_folder+'s1_data_raw.mat')    
    spike_times=data['spike_times'] #Load spike times of all neurons
    vels=data['vels'] #Load x and y velocities
    vel_times=data['vel_times'] #Load times at which velocities were recorded

if dataset=='m1':    
    data=io.loadmat(load_folder+'m1_data_raw.mat')    
    spike_times=data['spike_times'] #Load spike times of all neurons
    vels=data['vels'] #Load x and y velocities
    vel_times=data['vel_times'] #Load times at which velocities were recorded    
    
if dataset=='hc':
    data=io.loadmat(load_folder+'hc_data_raw.mat')
    spike_times=data['spike_times'] #Load spike times of all neurons
    pos=data['pos'] #Load x and y positions
    pos_times=data['pos_times'][0] #Load times at which positions were recorded    


## 3. Preprocess Data

Total amount of time to use spikes from  

In [8]:
if dataset=='s1':
    t_total=.6 #We use 600 ms of neural activity surrounding the output

if dataset=='m1':
    t_total=.6 #We use 600 ms of neural activity leading up to the output

if dataset=='hc':
    t_total=2 #We use 2 sec of neural activity surrounding the output

Options for extracting the data (these are copied from the preprocessing file)

In [9]:
if dataset=='s1' or dataset=='m1':
    t_start=vel_times[0] #Time to start extracting data - here the first time velocity was recorded
    t_end=vel_times[-1] #Time to finish extracting data - here the last time velocity was recorded
    
if dataset=='hc':    
    t_start=pos_times[0] #Time to start extracting data - here the first time position was recorded
    t_end=5608
    
downsample_factor=1 #Downsampling of output (to make binning go faster). 1 means no downsampling.


#When loading the Matlab cell "spike_times", Python puts it in a format with an extra unnecessary dimension
#First, we will put spike_times in a cleaner format: an array of arrays
spike_times=np.squeeze(spike_times)
for i in range(spike_times.shape[0]):
    spike_times[i]=np.squeeze(spike_times[i])

Set the bin sizes we will test

In [10]:
if dataset=='s1' or dataset=='m1':
    dts=[.01,.02,.03,.04,.05,.1] #Size of time bins (in seconds)

if dataset=='hc':
    dts=[.03,.05,.1,.2,.4] #Size of time bins (in seconds)

num_folds=len(dts) #Number of loops we'll do (I'm just calling it "folds" so I can keep old code that used CV folds)

Initializations of lists/vectors

In [11]:
mean_r2_wf=np.empty(num_folds)
mean_r2_wc=np.empty(num_folds)
mean_r2_xgb=np.empty(num_folds)
mean_r2_dnn=np.empty(num_folds)
mean_r2_rnn=np.empty(num_folds)
mean_r2_gru=np.empty(num_folds)
mean_r2_lstm=np.empty(num_folds)
mean_r2_svr=np.empty(num_folds)

y_test_all=[]
y_train_all=[]
y_valid_all=[]

y_pred_wf_all=[]
y_pred_wc_all=[]
y_pred_xgb_all=[]
y_pred_dnn_all=[]
y_pred_rnn_all=[]
y_pred_gru_all=[]
y_pred_lstm_all=[]
y_pred_svr_all=[]

## Loop over bin sizes and do everything!

In [12]:
for i in range(num_folds): #loop over bin sizes
    dt=dts[i] #Get the bin size for the current loop
    
    
    #### FORMAT OUTPUT ####
    
    #Bin output (velocity) data using "bin_output" function
    if dataset=='s1' or dataset=='m1':
        vels_binned=bin_output(vels,vel_times,dt,t_start,t_end,downsample_factor)
        y=vels_binned
    if dataset=='hc':
        pos_binned=bin_output(pos,pos_times,dt,t_start,t_end,downsample_factor)
        y=pos_binned
    
    
    
    #### FORMAT INPUT ####
    
    #Bin neural data using "bin_spikes" function
    neural_data=bin_spikes(spike_times,dt,t_start,t_end)
    
    #Remove neurons with too few spikes in HC dataset
    if dataset=='hc':
        nd_sum=np.nansum(neural_data,axis=0)
        rmv_nrn=np.where(nd_sum<100)
        neural_data=np.delete(neural_data,rmv_nrn,1)
    
    # Define what bins to use spikes from (with respect to the output)    
    if dataset=='s1' or dataset=='hc':
        bins_total=int(t_total/dt) #How many total bins should be used     
        bins_before=int(np.floor((bins_total-1)/2.)) #How many bins of neural data prior to the output are used for decoding (half of bins_total, rounded down)        
        bins_current=1 #Whether to use concurrent time bin of neural data        
        bins_after=int(np.ceil((bins_total-1)/2.)) #How many bins of neural data after the output are used for decoding (half of bins_total, rounded up)

    if dataset=='m1':
        bins_total=int(t_total/dt) #How many total bins should be used 
        bins_before=bins_total-1 #How many bins of neural data prior to the output are used for decoding
        bins_current=1 #Whether to use concurrent time bin of neural data 
        bins_after=0 #How many bins of neural data after the output are used for decoding
        
    print(bins_before,bins_current,bins_after)
                
    # Format input covariate matrix for recurrent neural networks (SimpleRNN, GRU, LSTM)
    # Function to get the covariate matrix that includes spike history from previous bins
    X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

    # Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
    #Put in "flat" format, so each "neuron / time" is a single feature
    X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))
    
    
    # In HC dataset, remove time bins with no output (y value)
    if dataset=='hc':
        #Remove time bins with no output (y value)
        rmv_time=np.where(np.isnan(y[:,0]) | np.isnan(y[:,1]))
        X=np.delete(X,rmv_time,0)
        X_flat=np.delete(X_flat,rmv_time,0)
        y=np.delete(y,rmv_time,0)
        
        
        
    ###### Define training/testing/validation sets ######

    if dataset=='hc':

        #Size of sets
        test_size=int(450/dt) #7.5 min
        valid_size=test_size #validation size is the same as the test size
        train_size=int(2250/dt) #37.5 min

        #End indices
        end_idx=np.int(X.shape[0]*.8) #End of test set
        tr_end_idx=end_idx-test_size-valid_size #End of training set

    if dataset=='s1':
        #Size of sets
        test_size=int(300/dt) #5 min
        valid_size=test_size #validation size is the same as the test size
        train_size=int(1200/dt) # 20 min

        #End indices
        end_idx=np.int(X.shape[0]*.9)#End of test set
        tr_end_idx=end_idx-2*test_size #End of training set

    if dataset=='m1':
        #Size of sets
        test_size=int(300/dt) #5 min
        valid_size=test_size #validation size is the same as the test size
        train_size=int(600/dt) # 10 min

        #End indices
        end_idx=np.int(X.shape[0]*1)#End of test set
        tr_end_idx=end_idx-2*test_size #End of training set


    #Range of sets
    testing_range=[end_idx-test_size,end_idx] #Testing set (length of test_size, goes up until end_idx)
    valid_range=[end_idx-test_size-valid_size,end_idx-test_size] #Validation set (length of valid_size, goes up until beginning of test set)
    training_range=[tr_end_idx-train_size,tr_end_idx] #Training set (length of train_size, goes up until beginning of validation set)



 

    ###### RUN DECODERS #######
    
    t1=time.time()

    num_examples=X.shape[0]


    ######### SPLIT DATA INTO TRAINING/TESTING/VALIDATION #########
    

    #Note that all sets have a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
    #This makes it so that the different sets don't include overlapping neural data
    
    #Testing set
    testing_set=np.arange(testing_range[0]+bins_before,testing_range[1]-bins_after)

    #Validation set
    valid_set=np.arange(valid_range[0]+bins_before,valid_range[1]-bins_after)

    #Training_set
    training_set=np.arange(training_range[0]+bins_before,training_range[1]-bins_after)

                
    #Get training data
    X_train=X[training_set,:,:]
    X_flat_train=X_flat[training_set,:]
    y_train=y[training_set,:]
    
    #Get testing data
    X_test=X[testing_set,:,:]
    X_flat_test=X_flat[testing_set,:]
    y_test=y[testing_set,:]

    #Get validation data
    X_valid=X[valid_set,:,:]
    X_flat_valid=X_flat[valid_set,:]
    y_valid=y[valid_set,:]

    
    #Preprocess data
    #Z-score "X" inputs. 
    X_train_mean=np.nanmean(X_train,axis=0)
    X_train_std=np.nanstd(X_train,axis=0)
    X_train=(X_train-X_train_mean)/X_train_std
    X_test=(X_test-X_train_mean)/X_train_std
    X_valid=(X_valid-X_train_mean)/X_train_std

    #Z-score "X_flat" inputs. 
    X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
    X_flat_train_std=np.nanstd(X_flat_train,axis=0)
    X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
    X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std
    X_flat_valid=(X_flat_valid-X_flat_train_mean)/X_flat_train_std

    #Zero-center outputs
    y_train_mean=np.nanmean(y_train,axis=0)
    y_train=y_train-y_train_mean
    y_test=y_test-y_train_mean
    y_valid=y_valid-y_train_mean
    
    #Z-score outputs (for SVR)
    y_train_std=np.nanstd(y_train,axis=0)
    y_zscore_train=y_train/y_train_std
    y_zscore_test=y_test/y_train_std
    y_zscore_valid=y_valid/y_train_std    
    
    
    
    ###### DECODING ###
    y_test_all.append(y_test)
    y_train_all.append(y_train)
    y_valid_all.append(y_valid)


    
    ###### WIENER FILTER ###
    if run_wf:
        
        #Declare model
        model_wf=WienerFilterDecoder()
        #Fit model
        model_wf.fit(X_flat_train,y_train)
        #Get predictions
        y_test_predicted_wf=model_wf.predict(X_flat_test)   
        #Get metric of fit
        mean_r2_wf[i]=np.mean(get_R2(y_test,y_test_predicted_wf))

        R2s_wf=get_R2(y_test,y_test_predicted_wf)
        print('R2s_wf:', R2s_wf)
    
        y_pred_wf_all.append(y_test_predicted_wf)
        
    ##### WIENER CASCADE #####
    if run_wc:
        
    #Get hyperparameters using validation set
    
        def wc_evaluate(degree):
            model_wc=WienerCascadeDecoder(degree)
            model_wc.fit(X_flat_train,y_train)
            y_valid_predicted_wc=model_wc.predict(X_flat_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_wc))

        wcBO = BayesianOptimization(wc_evaluate, {'degree': (1, 5.01)}, verbose=0)    
        wcBO.maximize(init_points=3, n_iter=3)
        best_params=wcBO.res['max']['max_params']
        degree=best_params['degree']
        print("degree=", degree)

        # Run model w/ above hyperparameters
        model_wc=WienerCascadeDecoder(degree)
        model_wc.fit(X_flat_train,y_train)
        y_test_predicted_wc=model_wc.predict(X_flat_test)
        mean_r2_wc[i]=np.mean(get_R2(y_test,y_test_predicted_wc))    

        R2s_wc=get_R2(y_test,y_test_predicted_wc)
        print('R2s_wc:', R2s_wc)
    
        y_pred_wc_all.append(y_test_predicted_wc)
    
    ##### SIMPLE RNN ######
    if run_rnn:
        
        #Get hyperparameters using validation set

        def rnn_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_rnn=SimpleRNNDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
            model_rnn.fit(X_train,y_train)
            y_valid_predicted_rnn=model_rnn.predict(X_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_rnn))

        rnnBO = BayesianOptimization(rnn_evaluate, {'num_units': (50, 600), 'frac_dropout': (0,.5), 'n_epochs': (2,21)})
        rnnBO.maximize(init_points=20, n_iter=20, kappa=10)
        best_params=rnnBO.res['max']['max_params']

        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=np.int(best_params['n_epochs'])
        num_units=np.int(best_params['num_units'])

        # Run model w/ above hyperparameters
        model_rnn=SimpleRNNDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
        model_rnn.fit(X_train,y_train)
        y_test_predicted_rnn=model_rnn.predict(X_test)
        mean_r2_rnn[i]=np.mean(get_R2(y_test,y_test_predicted_rnn))    

        R2s_rnn=get_R2(y_test,y_test_predicted_rnn)
        print('R2s:', R2s_rnn)
    
        y_pred_rnn_all.append(y_test_predicted_rnn)
    
    
    ##### GRU ######
    if run_gru:
        
        #Get hyperparameters using validation set
        def gru_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_gru=GRUDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
            model_gru.fit(X_train,y_train)
            y_valid_predicted_gru=model_gru.predict(X_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_gru))

        gruBO = BayesianOptimization(gru_evaluate, {'num_units': (50, 600), 'frac_dropout': (0,.5), 'n_epochs': (2,21)})
        gruBO.maximize(init_points=20, n_iter=20,kappa=10)
        best_params=gruBO.res['max']['max_params']

        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=np.int(best_params['n_epochs'])
        num_units=np.int(best_params['num_units'])

        # Run model w/ above hyperparameters
        model_gru=GRUDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
        model_gru.fit(X_train,y_train)
        y_test_predicted_gru=model_gru.predict(X_test)
        mean_r2_gru[i]=np.mean(get_R2(y_test,y_test_predicted_gru))    

        R2s_gru=get_R2(y_test,y_test_predicted_gru)
        print('R2s:', R2s_gru)
    
        y_pred_gru_all.append(y_test_predicted_gru)


    
    
    ##### LSTM ######
    if run_lstm:
        
        #Get hyperparameters using validation set
        def lstm_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_lstm=LSTMDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
            model_lstm.fit(X_train,y_train)
            y_valid_predicted_lstm=model_lstm.predict(X_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_lstm))

        lstmBO = BayesianOptimization(lstm_evaluate, {'num_units': (50, 600), 'frac_dropout': (0,.5), 'n_epochs': (2,21)})
        lstmBO.maximize(init_points=20, n_iter=20, kappa=10)
        best_params=lstmBO.res['max']['max_params']

        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=np.int(best_params['n_epochs'])
        num_units=np.int(best_params['num_units'])

        # Run model w/ above hyperparameters
        model_lstm=LSTMDecoder(units=num_units,dropout=frac_dropout,num_epochs=n_epochs)
        model_lstm.fit(X_train,y_train)
        y_test_predicted_lstm=model_lstm.predict(X_test)
        mean_r2_lstm[i]=np.mean(get_R2(y_test,y_test_predicted_lstm))    

        R2s_lstm=get_R2(y_test,y_test_predicted_lstm)
        print('R2s:', R2s_lstm)   

        y_pred_lstm_all.append(y_test_predicted_lstm)

    
    
    
    
    ##### Dense (Feedforward) NN ######
    if run_dnn:
        
        #Get hyperparameters using validation set
        def dnn_evaluate(num_units,frac_dropout,n_epochs):
            num_units=int(num_units)
            frac_dropout=float(frac_dropout)
            n_epochs=int(n_epochs)
            model_dnn=DenseNNDecoder(units=[num_units,num_units],dropout=frac_dropout,num_epochs=n_epochs)
            model_dnn.fit(X_flat_train,y_train)
            y_valid_predicted_dnn=model_dnn.predict(X_flat_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_dnn))

        dnnBO = BayesianOptimization(dnn_evaluate, {'num_units': (50, 600), 'frac_dropout': (0,.5), 'n_epochs': (2,21)})
        dnnBO.maximize(init_points=20, n_iter=20, kappa=10)
        best_params=dnnBO.res['max']['max_params']

        frac_dropout=float(best_params['frac_dropout'])
        n_epochs=np.int(best_params['n_epochs'])
        num_units=np.int(best_params['num_units'])

        # Run model w/ above hyperparameters
        model_dnn=DenseNNDecoder(units=[num_units,num_units],dropout=frac_dropout,num_epochs=n_epochs)
        model_dnn.fit(X_flat_train,y_train)
        y_test_predicted_dnn=model_dnn.predict(X_flat_test)
        mean_r2_dnn[i]=np.mean(get_R2(y_test,y_test_predicted_dnn))    

        R2s_dnn=get_R2(y_test,y_test_predicted_dnn)
        print('R2s:', R2s_dnn)    
    
        y_pred_dnn_all.append(y_test_predicted_dnn)


        
        
    ##### SVR #####
    if run_svr:
        
        #Get hyperparameters using validation set
        max_iter=4000 #2000 for M1, 4000 for HC

        def svr_evaluate(C):
            model_svr=SVRDecoder(C=C, max_iter=max_iter)
            model_svr.fit(X_flat_train,y_zscore_train)
            y_valid_predicted_svr=model_svr.predict(X_flat_valid)
            return np.mean(get_R2(y_zscore_valid,y_valid_predicted_svr))

        svrBO = BayesianOptimization(svr_evaluate, {'C': (.5, 10)}, verbose=0)    
        svrBO.maximize(init_points=5, n_iter=5)
        best_params=svrBO.res['max']['max_params']
        C=best_params['C']
        print("C=", C)

        # Run model w/ above hyperparameters
        model_svr=SVRDecoder(C=C, max_iter=max_iter)
        model_svr.fit(X_flat_train,y_zscore_train)
        y_test_predicted_svr=model_svr.predict(X_flat_test)
        mean_r2_svr[i]=np.mean(get_R2(y_zscore_test,y_test_predicted_svr))    

        R2s_svr=get_R2(y_zscore_test,y_test_predicted_svr)
        print('R2s_svr:', R2s_svr)    
    
        y_pred_svr_all.append(y_test_predicted_svr)



    ##### XGBOOST ######
    if run_xgb:
        
        
        #Get hyperparameters using validation set
        def xgb_evaluate(max_depth,num_round,eta):
            max_depth=int(max_depth)
            num_round=int(num_round)
            eta=float(eta)
            model_xgb=XGBoostDecoder(max_depth=max_depth, num_round=num_round, eta=eta)
            model_xgb.fit(X_flat_train,y_train)
            y_valid_predicted_xgb=model_xgb.predict(X_flat_valid)
            return np.mean(get_R2(y_valid,y_valid_predicted_xgb))

        xgbBO = BayesianOptimization(xgb_evaluate, {'max_depth': (2, 10.01), 'num_round': (100,700), 'eta': (0, 1)})


        xgbBO.maximize(init_points=20, n_iter=20, kappa=10)
        best_params=xgbBO.res['max']['max_params']

        num_round=np.int(best_params['num_round'])
        max_depth=np.int(best_params['max_depth'])
        eta=best_params['eta']
    
        # Run model w/ above hyperparameters
        model_xgb=XGBoostDecoder(max_depth=max_depth, num_round=num_round, eta=eta)
        model_xgb.fit(X_flat_train,y_train)
        y_test_predicted_xgb=model_xgb.predict(X_flat_test)
        mean_r2_xgb[i]=np.mean(get_R2(y_test,y_test_predicted_xgb))    

        R2s_xgb=get_R2(y_test,y_test_predicted_xgb)
        print('R2s:', R2s_xgb)
    
        y_pred_xgb_all.append(y_test_predicted_xgb)        
        
    
    print ("\n")
      
    
    time_elapsed=time.time()-t1

    ###### SAVE RESULTS #####
    #Note that I save them after every loop rather than at the end in case the code/computer crashes for some reason while running    
    if run_wf:
        with open(save_folder+dataset+'_results_binsize_wf.pickle','wb') as f:
            pickle.dump([mean_r2_wf,y_pred_wf_all],f)

    if run_wc:
        with open(save_folder+dataset+'_results_binsize_wc.pickle','wb') as f:
            pickle.dump([mean_r2_wc,y_pred_wc_all],f)

    if run_xgb:
        with open(save_folder+dataset+'_results_binsize_xgb.pickle','wb') as f:
            pickle.dump([mean_r2_xgb,y_pred_xgb_all],f)

    if run_dnn:
        with open(save_folder+dataset+'_results_binsize_dnn.pickle','wb') as f:
            pickle.dump([mean_r2_dnn,y_pred_dnn_all,time_elapsed],f)

    if run_rnn:
        with open(save_folder+dataset+'_results_binsize_rnn.pickle','wb') as f:
            pickle.dump([mean_r2_rnn,y_pred_rnn_all,time_elapsed],f)

    if run_gru:
        with open(save_folder+dataset+'_results_binsize_gru.pickle','wb') as f:
            pickle.dump([mean_r2_gru,y_pred_gru_all,time_elapsed],f)

    if run_lstm:
        with open(save_folder+dataset+'_results_binsize_lstm.pickle','wb') as f:
            pickle.dump([mean_r2_lstm,y_pred_lstm_all,time_elapsed],f)

    if run_svr:
        with open(save_folder+dataset+'_results_binsize_svr.pickle','wb') as f:
            pickle.dump([mean_r2_svr,y_pred_svr_all,time_elapsed],f)
   
    
# print("time_elapsed:",time_elapsed)

#Save ground truth results
with open(save_folder+dataset+'_ground_truth_binsize.pickle','wb') as f:
    pickle.dump([y_test_all,y_train_all,y_valid_all],f)

(32, 1, 33)
('R2s_wf:', array([0.29717957, 0.38378935]))


(19, 1, 20)
('R2s_wf:', array([0.29388755, 0.37917592]))


(9, 1, 10)
('R2s_wf:', array([0.29808547, 0.37770163]))


(4, 1, 5)
('R2s_wf:', array([0.30583955, 0.38009328]))


(2, 1, 2)
('R2s_wf:', array([0.31973523, 0.38765108]))


